# Arabic Font Classification
This notebook is insprired from Mahmoud Aslan's work on Arabic Font Classification.

## Imports

In [1]:
import transformers
import torch
torch.cuda.is_available()

True

In [2]:
import numpy as np
import random as rn
import os
import tensorflow as tf

#gpu tensorflow
tf.config.experimental.set_visible_devices([], 'GPU')

rn.seed(42)
np.random.seed(42)
tf.random.set_seed(42)
rng = tf.random.experimental.Generator.from_seed(1234)

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED']=str(0)
# os.environ['CUDA_VISIBLE_DEVICES'] = ''


## Load data

In [4]:
from pathlib import Path
RUFA_PATH = Path('../data/rufa/')

In [5]:
synth_dir = Path.joinpath(RUFA_PATH, 'synth')
real_dir = Path.joinpath(RUFA_PATH, 'real')

CLASS_NAMES = np.array([item.name for item in real_dir.glob('*')])
CLASS_NAMES

array(['farsi', 'ruqaa'], dtype='<U5')

In [6]:
synth_paths = tf.data.Dataset.list_files(str(synth_dir / '*/*.jpg'), seed=42)
real_paths = tf.data.Dataset.list_files(str(real_dir / '*/*.jpg'), seed=42)

## Train, val, mismatch, test split

In [7]:
_max_data_size = 2**32
_test_ratio = 0.2


def test_set_check(item):
    id = tf.strings.split(tf.strings.split(item, os.sep)[-1], '.')[0]
    hash = tf.strings.to_hash_bucket_fast(id, _max_data_size)
    return tf.cast(hash, tf.float64) < _test_ratio * _max_data_size

def train_set_check(item):
    id = tf.strings.split(tf.strings.split(item, os.sep)[-1], '.')[0]
    hash = tf.strings.to_hash_bucket_fast(id, _max_data_size)
    return tf.cast(hash, tf.float64) >= _test_ratio * _max_data_size

In [8]:
train_paths = synth_paths.filter(train_set_check)
val_paths = synth_paths.filter(test_set_check)
mismatch_paths = real_paths.filter(test_set_check)
test_paths = real_paths.filter(train_set_check)

## Preprocessing

In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32
mapping = {0: 'farsi', 1: 'ruqaa'}

def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  ds = ds.shuffle(buffer_size=shuffle_buffer_size, seed=42)
  ds = ds.batch(BATCH_SIZE)
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

def parse_image(data_instance):
  parts = tf.strings.split(data_instance, os.sep)
  label = tf.cast(tf.argmax(tf.cast(parts[-2] == np.array(list(mapping.values())), dtype=tf.float16)), tf.float16)

  image = tf.io.read_file(data_instance)
  image = tf.image.decode_jpeg(image, 1)
  image = tf.image.convert_image_dtype(image, tf.float32)

  if parts[-3] == 'synth':
    noise = rng.normal(shape=tf.shape(image), mean=0.0, stddev=0.015, dtype=tf.float32)
    image = tf.add( image, noise)
    image = tf.clip_by_value(image, 0.0, 1.0)

    image = tf.image.adjust_jpeg_quality(image, 90)

  return image, label

In [10]:
train_ds = train_paths.map(parse_image)
val_ds = val_paths.map(parse_image)
mismatch_ds = mismatch_paths.map(parse_image)
full_train_ds = train_ds.concatenate(val_ds.concatenate(mismatch_ds))
test_ds = test_paths.map(parse_image)

train_ds = prepare_for_training(train_ds)
val_ds = prepare_for_training(val_ds)
mismatch_ds = prepare_for_training(mismatch_ds)
full_train_ds = prepare_for_training(full_train_ds)
test_ds = prepare_for_training(test_ds)

## Model

In [11]:
def cnn(input_shape, output_shape):
    num_classes = output_shape[0]
    dropout_seed = 708090
    kernel_seed = 42
  
    model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape=input_shape, kernel_initializer=tf.keras.initializers.GlorotUniform(seed=kernel_seed)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Dropout(0.1, seed=dropout_seed),
      tf.keras.layers.Conv2D(32, 5, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=kernel_seed)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Dropout(0.1, seed=dropout_seed),
      tf.keras.layers.Conv2D(64, 10, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=kernel_seed)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Dropout(0.1, seed=dropout_seed),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu', kernel_regularizer='l2', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=kernel_seed)),
      tf.keras.layers.Dropout(0.2, seed=dropout_seed),
      tf.keras.layers.Dense(16, activation='relu', kernel_regularizer='l2', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=kernel_seed)),
      tf.keras.layers.Dropout(0.2, seed=dropout_seed),
      tf.keras.layers.Dense(num_classes, activation='sigmoid', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=kernel_seed))
    ])

    return model

In [12]:
epochs = 6
callbacks = None

In [13]:
model = cnn((100, 100, 1), (1,))
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer='Adam', metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 49, 49, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 45, 45, 32)        12832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 22, 22, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 22, 22, 32)        0

## Training and evaluation

In [14]:
model.fit(
  full_train_ds,
  epochs=epochs,
  callbacks=callbacks
)

Epoch 1/6
1254/1254 [==============================] - 667s 530ms/step - loss: 0.3897 - accuracy: 0.8245
Epoch 2/6
  79/1254 [>.............................] - ETA: 13:26 - loss: 0.0787 - accuracy: 0.9885

KeyboardInterrupt: 

#### Evaluate

In [ ]:
test_score = model.evaluate(test_ds)
print(f"Test score: {test_score}")

14/14 [==============================] - 1s 99ms/step - loss: 0.2316 - accuracy: 0.9712
Test score: [0.2316255122423172, 0.971222996711731]


In [ ]:
assert np.allclose(test_score, [.231625, .971222])